<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [ ]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [ ]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [ ]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [ ]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

In [ ]:
 TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [ ]:
import pdb; 
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = dropout ,bidirectional = bidirectional)  # YOUR CODE GOES HERE
        
        self.cls = nn.Linear(2*hidden_dim, 1)

        self.dropout = nn.Dropout(p=dropout)

        self.fc = nn.Sigmoid()  # YOUR CODE GOES HERE
        
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        #pdb.set_trace()
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = torch.cat((hidden[-2,:,:],hidden[-1,:,:]), 1)  # YOUR CODE GOES HERE
        hidden = self.dropout(hidden) 
        hidden = self.cls(hidden)
        
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
            
        return hidden.reshape(-1)

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = False
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

NameError: ignored

In [ ]:
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCELoss()

max_epochs = 20

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [ ]:
import numpy as np
#%debug
min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      input_embeds = batch._get_field_values(['text'])[0].to(device)
      input_text_lengths = batch._get_field_values(['text'])[1].cpu().data
      labels = batch._get_field_values(['label']).to(device)


      optimizer.zero_grad()
      prediction = model(input_embeds, input_text_lengths)
      loss = criterion(prediction, labels)
      loss.backward()
      optimizer.step()
      train_loss+=loss
      pbar.set_description(f"Epoch {epoch }. Train Loss: {loss:.4}")

            

        #YOUR CODE GOES HERE

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      input_embeds = batch._get_field_values(['text'])[0].to(device)
      input_text_lengths = batch._get_field_values(['text'])[1].cpu().data
      labels = batch._get_field_values(['label']).to(device)

      prediction = model(input_embeds, input_text_lengths)
      loss = criterion(prediction, labels)
      val_loss+=loss
      pbar.set_description(f"Epoch {epoch}. Train Loss: {loss:.4}")

        # YOUR CODE GOES HERE

    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.6568645238876343, Validation Loss: 0.662019670009613


Epoch: 2, Training Loss: 0.5901405215263367, Validation Loss: 0.569666862487793


Epoch: 3, Training Loss: 0.46286678314208984, Validation Loss: 0.5523691773414612


Epoch: 4, Training Loss: 0.3234744966030121, Validation Loss: 0.532975971698761


Epoch: 5, Training Loss: 0.22455507516860962, Validation Loss: 0.5381262302398682


Epoch: 6, Training Loss: 0.1326291859149933, Validation Loss: 0.618437647819519


<All keys matched successfully>


Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**:

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
#%debug

model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
score_all = 0 

for it, batch in pbar:
      input_embeds = batch._get_field_values(['text'])[0].to(device)
      input_text_lengths = batch._get_field_values(['text'])[1].cpu().data
      labels = batch._get_field_values(['label']).to(device)

      prediction = model(input_embeds, input_text_lengths)

      answ = [1 if i > 0 else 0 for i in prediction ] 
      score = f1_score(labels.cpu().data, answ, average='macro') 
      score_all+=score
      pbar.set_description(f"Epoch {epoch + 1}. Train Score: {score:.4}")

        # YOUR CODE GOES HERE

print(score_all/len(test_iter))

0.7135867131728709


## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [ ]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [ ]:
import pdb; 

class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Conv1d(emb_dim, out_channels, kernel_size=kernel_sizes[0], padding=1, stride=2)  # YOUR CODE GOES HERE
        
        self.conv_1 = nn.Conv1d(emb_dim, out_channels, kernel_size=kernel_sizes[1], padding=1, stride=2)  # YOUR CODE GOES HERE
        
        self.conv_2 = nn.Conv1d(emb_dim, out_channels, kernel_size=kernel_sizes[2], padding=1, stride=2)  # YOUR CODE GOES HERE
        
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        embedded = embedded.permute(0, 2, 1)  # may be reshape here
        
        conved_0 = F.relu(self.conv_0(embedded))  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded))  # may be reshape here
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
        #pdb.set_trace()
        return self.fc(cat).reshape(-1)

In [ ]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=64
dropout = 0.5
dim = 300

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [ ]:
model.to(device)

CNN(
  (embedding): Embedding(202341, 300)
  (conv_0): Conv1d(300, 64, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv_1): Conv1d(300, 64, kernel_size=(4,), stride=(2,), padding=(1,))
  (conv_2): Conv1d(300, 64, kernel_size=(5,), stride=(2,), padding=(1,))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [ ]:
max_epochs = 30

Обучите!

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0
patience = 2

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      input_embeds = batch._get_field_values(['text'])
      labels = batch._get_field_values(['label'])


      optimizer.zero_grad()
      prediction = model(input_embeds)
      loss = criterion(prediction, labels)
      loss.backward()
      optimizer.step()
      train_loss+=loss
      pbar.set_description(f"Epoch {epoch }. Train Loss: {loss:.4}")      

        #YOUR CODE GOES HERE

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():
      for it, batch in pbar:
        input_embeds = batch._get_field_values(['text'])
        labels = batch._get_field_values(['label'])

        prediction = model(input_embeds)
        loss = criterion(prediction, labels)
        val_loss+=loss
        pbar.set_description(f"Epoch {epoch}. Train Loss: {loss:.4}")
      
        # YOUR CODE GOES HERE
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.6711640357971191, Validation Loss: 0.5286245346069336


Epoch: 2, Training Loss: 0.5265653729438782, Validation Loss: 0.46443548798561096


Epoch: 3, Training Loss: 0.45008060336112976, Validation Loss: 0.4227939248085022


Epoch: 4, Training Loss: 0.3709670901298523, Validation Loss: 0.3909359872341156


Epoch: 5, Training Loss: 0.29709941148757935, Validation Loss: 0.3691879212856293


Epoch: 6, Training Loss: 0.2224307358264923, Validation Loss: 0.35887134075164795


Epoch: 7, Training Loss: 0.15767145156860352, Validation Loss: 0.3670504093170166


<All keys matched successfully>

In [ ]:
from sklearn.metrics import f1_score

model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
score_all = 0 

for it, batch in pbar:
  with torch.no_grad():
      input_embeds = batch._get_field_values(['text'])
      labels = batch._get_field_values(['label'])

      prediction = model(input_embeds)

      answ = [1 if i > 0 else 0 for i in prediction ] 
      score = f1_score(labels.cpu().data, answ, average='macro') 
      score_all+=score
      pbar.set_description(f"Epoch {epoch + 1}. Train Score: {score:.4}")

        # YOUR CODE GOES HERE

print(score_all/len(test_iter))

0.45936825487723976


Посчитайте f1-score вашего классификатора.

**Ответ**:

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [ ]:
!pip install -q captum

     |████████████████████████████████| 4.4MB 4.2MB/s 


In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [ ]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [ ]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 1.00 ) , delta:  tensor([9.2492e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.43 ) , delta:  tensor([7.0365e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.91 ) , delta:  tensor([7.5289e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.04 ) , delta:  tensor([6.2367e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.14 ) , delta:  tensor([6.2149e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.02 ) , delta:  tensor([3.4920e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,0.98,It was a fantastic performance ! pad
pos,neg (0.43),pos,0.83,Best film ever pad pad pad pad
pos,pos (0.91),pos,0.95,Such a great show! pad pad pad
neg,neg (0.04),pos,-0.60,It was a horrible movie pad pad
neg,neg (0.14),pos,-0.30,I've never watched something as bad pad
neg,neg (0.02),pos,-0.55,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,0.98,It was a fantastic performance ! pad
pos,neg (0.43),pos,0.83,Best film ever pad pad pad pad
pos,pos (0.91),pos,0.95,Such a great show! pad pad pad
neg,neg (0.04),pos,-0.60,It was a horrible movie pad pad
neg,neg (0.14),pos,-0.30,I've never watched something as bad pad
neg,neg (0.02),pos,-0.55,It is a disgusting movie! pad pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [ ]:
TEXT.build_vocab(trn, vectors="glove.twitter.27B.50d")# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)

word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

.vector_cache/glove.twitter.27B.zip: 1.52GB [04:49, 5.26MB/s]                            
100%|█████████▉| 1191619/1193514 [00:54<00:00, 23352.07it/s]

In [ ]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

100%|█████████▉| 1191619/1193514 [01:10<00:00, 23352.07it/s]

In [ ]:
dim = model.embedding.weight.shape[1]


model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

model.embedding.weight = nn.Parameter(word_embeddings)

assert prev_shape == model.embedding.weight.shape
model.to(device)

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()
max_epochs=20

Вы знаете, что делать.

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0
patience=1

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
      input_embeds = batch._get_field_values(['text'])
      labels = batch._get_field_values(['label'])
      optimizer.zero_grad()
      prediction = model(input_embeds)
      loss = criterion(prediction, labels)
      loss.backward()
      optimizer.step()
      train_loss+=loss
      pbar.set_description(f"Epoch {epoch }. Train Loss: {loss:.4}")      


    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():
      for it, batch in pbar:
        input_embeds = batch._get_field_values(['text'])
        labels = batch._get_field_values(['label'])

        prediction = model(input_embeds)
        loss = criterion(prediction, labels)
        val_loss+=loss
        pbar.set_description(f"Epoch {epoch}. Train Loss: {loss:.4}")
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.6589811444282532, Validation Loss: 0.5577313899993896


Epoch: 2, Training Loss: 0.4696713089942932, Validation Loss: 0.38647618889808655


Epoch: 3, Training Loss: 0.3516860902309418, Validation Loss: 0.3454751968383789


Epoch: 4, Training Loss: 0.26671478152275085, Validation Loss: 0.3199261426925659


Epoch: 5, Training Loss: 0.18596690893173218, Validation Loss: 0.3099309206008911


<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**:

In [ ]:
from sklearn.metrics import f1_score

model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
score_all = 0 

for it, batch in pbar:
  with torch.no_grad():
      input_embeds = batch._get_field_values(['text'])
      labels = batch._get_field_values(['label'])

      prediction = model(input_embeds)

      answ = [1 if i > 0 else 0 for i in prediction ] 
      score = f1_score(labels.cpu().data, answ, average='macro') 
      score_all+=score
      pbar.set_description(f"Epoch {epoch + 1}. Train Score: {score:.4}")

        # YOUR CODE GOES HERE

print(score_all/len(test_iter))

0.4658021192912851


Проверим насколько все хорошо!

In [ ]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 1.00 ) , delta:  tensor([1.9328e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.38 ) , delta:  tensor([4.5523e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.94 ) , delta:  tensor([8.0505e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.23 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([1.5758e-05], device='cuda:0', dtype=torch.float64)


In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.22,It was a fantastic performance ! pad
pos,neg (0.38),pos,1.67,Best film ever pad pad pad pad
pos,pos (0.94),pos,1.32,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.99,It was a horrible movie pad pad
neg,neg (0.23),pos,-0.27,I've never watched something as bad pad
neg,neg (0.01),pos,-0.69,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.22,It was a fantastic performance ! pad
pos,neg (0.38),pos,1.67,Best film ever pad pad pad pad
pos,pos (0.94),pos,1.32,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.99,It was a horrible movie pad pad
neg,neg (0.23),pos,-0.27,I've never watched something as bad pad
neg,neg (0.01),pos,-0.69,It is a disgusting movie! pad pad
